In [1]:
import sys
sys.path.append("/dccstor/srosent2/primeqa-mengxia/primeqa/primeqa/")
# print(sys.path)
from lfqa_kb.metrics.utils import _metric_max_over_ground_truths
import pandas as pd

In [2]:
# Gold Data

import json
ELI5 = {}

with open("/dccstor/srosent2/primeqa-mengxia/data/dpr-100passages_withkg_best_all_cased_pr/eli5-dev-kilt-dpr-kg-00.json",'r') as f:
    for line in f.readlines():
        data = json.loads(line)
        ELI5[data['id']] = data

In [3]:
# Gold Data

ASQA = {}

with open("/dccstor/srosent2/primeqa-mengxia/data/asqa/passages_kg_pr/asqa-dev-kg.json",'r') as f:
    for line in f.readlines():
        data = json.loads(line)
        ASQA[data['id']] = data

In [9]:
def rouge(ref_data, prediction_file):
    baseline_df = pd.read_json(prediction_file, lines=True, dtype=str)  #orient='records', lines=True
    baseline_df['hf_rouge'] = 0.0
    baseline_df['kilt_rouge'] = 0.0

    for i, row in baseline_df.iterrows():
        answers = []
        output = ref_data[row["id"]]['output']

        for o in output:
            if "answer" in o:
                answers.append(o["answer"])

        hf_rouge_score, kilt_rouge_score = _metric_max_over_ground_truths(row["text"],answers) # prediction_text
        
        baseline_df.loc[i, 'hf_rouge'] = hf_rouge_score
        baseline_df.loc[i, 'kilt_rouge'] = kilt_rouge_score
        
    hf_score = baseline_df['hf_rouge'].mean()
    kilt_score = baseline_df['kilt_rouge'].mean()
    return hf_score, kilt_score

In [5]:
hf_score, kilt_score = rouge("/dccstor/srosent2/primeqa-mengxia/experiments/eli5/filter_pr/all/oracle_train/kg_vocab/output/eval_predictions.json")
print(f"{hf_score} {kilt_score}")

TypeError: rouge() missing 1 required positional argument: 'prediction_file'

"/dccstor/srosent2/primeqa-mengxia/experiments/eli5/llm/*/*/*/predictions*.json"
"/dccstor/srosent2/primeqa-mengxia/experiments/eli5/default-cased/*/eval_predictions.json"

In [9]:
import glob

pred_files = glob.glob("/dccstor/srosent2/primeqa-mengxia/experiments/eli5/filter_pr/*/*/*/*/eval_predictions.json")

for file in pred_files:
    hf_score, kilt_score = rouge(file)
    print(f"{file[len('/dccstor/srosent2/primeqa-mengxia/experiments/eli5/'):]}|{hf_score}|{kilt_score}")

filter_pr/best/oracle_train/kg_vocab/output/eval_predictions.json|0.34091718714797986|0.3381247730560579
filter_pr/best/oracle_train/kg_triples/output/eval_predictions.json|0.3071780544589346|0.2777883819991962
filter_pr/best/oracle_train/kg_sentences/output/eval_predictions.json|0.2981136636605923|0.26207400629327754
filter_pr/best/classifier/kg_triples/output/eval_predictions.json|0.2989590024440737|0.2533164266857733
filter_pr/all/oracle_train/kg_vocab/output/eval_predictions.json|0.33947903100935856|0.3375327665511594
filter_pr/all/oracle_train/kg_triples/output/eval_predictions.json|0.30814305891866456|0.2801967707195021
filter_pr/all/oracle_train/kg_sentences/output/eval_predictions.json|0.2929297549962011|0.25826866264803655


In [20]:
import glob

pred_files = ["/dccstor/srosent2/primeqa-mengxia/experiments/eli5/default-cased/output/eval_predictions.json",
"/dccstor/srosent2/primeqa-mengxia/experiments/eli5/old/oracle-kg/default/output/eval_predictions.json"]

for file in pred_files:
    hf_score, kilt_score = rouge(ELI5, file)
    print(f"{file[len('/dccstor/srosent2/primeqa-mengxia/experiments/eli5/'):]}|{hf_score}|{kilt_score}")

default-cased/output/eval_predictions.json|0.29324764162032196|0.2603816447110422
old/oracle-kg/default/output/eval_predictions.json|0.30016056742210856|0.264891112224429


In [11]:
import glob

pred_files = glob.glob("/dccstor/srosent2/primeqa-mengxia/experiments/eli5/llm/*/*/*/predictions*.json")

for file in pred_files:
    hf_score, kilt_score = rouge(file)
    print(f"{file[len('/dccstor/srosent2/primeqa-mengxia/experiments/eli5/'):]}|{hf_score}|{kilt_score}")

llm/kg_vocab/google-flan-t5-xxl/prefix_default-passages_True-0shot_pktemp-1.0_5_0.7-nctx_3-minmaxtok_100_1024/predictions-0-1507.json|0.2017658339552232|0.1776883730223207
llm/kg_triples/google-flan-t5-xxl/prefix_default-passages_True-0shot_pktemp-1.0_5_0.7-nctx_3-minmaxtok_100_1024/predictions-0-1507.json|0.18500723552470225|0.17074034204820304
llm/kg_sentences/google-flan-t5-xxl/prefix_default-passages_True-0shot_pktemp-1.0_5_0.7-nctx_3-minmaxtok_100_1024/predictions-0-1507.json|0.1952088712029225|0.17302102435045755
llm/debug/google-flan-t5-small/prefix_default-passages_True-0shot_pktemp-0.25_100_1.0-nctx_3-minmaxtok_100_256/predictions-0-1507.json|nan|nan
llm/default/google-flan-t5-xxl/prefix_default-passages_True-0shot_pktemp-1.0_5_0.7-nctx_3-minmaxtok_100_1024/predictions-0-1507.json|0.20672720839226544|0.17312319014771305


In [5]:

# pred_files = glob.glob("/dccstor/srosent2/primeqa-mengxia/experiments/asqa/oracle_kg/all/kg_*/*/eval_predictions.json")

# for file in pred_files:
#     hf_score, kilt_score = rouge(ASQA, file)
#     print(f"{file[len('/dccstor/srosent2/primeqa-mengxia/experiments/eli5/'):]}|{hf_score}|{kilt_score}")

hf_score, kilt_score = rouge(ASQA, "/dccstor/srosent2/primeqa-mengxia/experiments/asqa/default-cased/output/eval_predictions.json")
print(f"default|{hf_score}|{kilt_score}")

ValueError: Expected object or value

oracle_kg/all/kg_vocab/output/eval_predictions.json|0.5131369113708163|0.4951915033863919
oracle_kg/all/kg_triples/output/eval_predictions.json|0.46579573056521606|0.4418892069347601
oracle_kg/all/kg_sentences/output/eval_predictions.json|0.4376146893600135|0.40684625414418885

In [11]:

import glob
pred_files = glob.glob("/dccstor/srosent2/generative/baseline_llms/ELI5-full/512-limit/closed-book/flan-t5-large/prefix_default-passages_False-0shot_pktemp-1.0_5_0.7-minmaxtok_100_1024/predictions-0-1507.json")

for file in pred_files:
    hf_score, kilt_score = rouge(ELI5, file)
    print(f"{file}|{hf_score}|{kilt_score}")

TypeError: cannot unpack non-iterable float object